FRI-based PCS

## How FRI works
A FRI protocol is a protocol for proving that a function $f : H \to \mathbb{F}$ is closed to a polynomial of low degree $d$, where $d \ll |H|$. 
The FRI PCS outputs the root of the Merkle tree as the commitment to the function $f$ while the FRI protocol consists of the evaluation proof.
The protocol can be divided into two phases: commit and query.
In the commit phash, the prover commits to (via Merkle trees) a series of functions generated from $f$ and random elements $v_0, v_1, ... $ from $\mathbb{K}$ provided by the verifier at each round. 
Then in the query phase, the prover provides a set of evaluations of the previously committed functions at a point randomly chosen by the verifier.

### Commit Phase
Denote by $p_0$ the function of low degree. 
In the commit phase, the polynomial $p_0$ is split into two polynomials $g_{0,1}, g_{0,2} : H^2 \to \mathbb{K}$ of degree lower than $d/2$ such that:
$$p_0(X) = g_{0,1}(X^2) + X g_{0,2}(X^2).$$

Then, the verifier sends a uniform randomness $v_0 \in \mathbb{K}$ and ask the prover to commit to the polynomial: $p_1(X) = g_{0,1}(X) + v_0 g_{0,2}(X)$.
Note that $p_1$ is a polynomial of degree less than $d/2$ and the commitment of $p_1$ is not over $H$ but over $H^2 = \{x^2 : x \in H \}$.

The prover then continues by splitting $p_1$ into $g_{1,1}$ and $g_{1,2}$ of degree lower than $d/4$, and constructs $p_2$ with a uniformly sampled $v_1 \in \mathbf{K}$ sent by the verifier.

This above interaction iterates $k = \log d$ times.
At last, $deg(p_k) = 0$ and the prover sends a constant $p_k$, representing a polynomial of degree lower than 1, to the verifier. 

### Query Phase
In the query phase, the verifier sends a randomness $r \in H$ to the prover and queries the evaluations $p_0(r), p_0(-r)$ and $p_1(r_2)$.
From $p_0(r), p_0(-r)$ the verifier computes $p_1(r^2)$ and checks that the computed value matches $p_1(r^2)$ sent by the prover.
In detail, $p_1(r^2)$ can be obtained as follows:
- $p_0(r) = g_{0,1}(r^2) + r\cdot g_{0,2}(r^2)$;
- $p_0(-r) = g_{0,1}(r^2) - r\cdot g_{0,2}(r^2)$;
- $p_1(r^2) = g_{0,1}(r^2) + v_0\cdot g_{0,2}(r^2)$.

In the next interaction, the verifier queries $p_1(-r^2)$ and $p_2(r^4)$ and checks the consistency between $p_1, p_2$ as before.
The interaction repeats till the constant $p_k$. 
The verifier checks that the value sent by the prover is indeed equal to the value that the verifier computed from the queries up until $p_{k-1}$.
To fully ensure correctness, the prover must accompany the evaluations that she sends with a claim of their existence (via Merkle tree paths).
Upon the completion of this process, the verifier has a first confirmation that the polynomials committed in the commit phase $p_0, p_1, . . . , p_k$ are consistent with each other.

Finally, to achieve the required bounds for the soundness of the protocol, the query phase is repeated multiple times.

### A Python example
At the beginning, we define two classes PolynomialOperations and MerkleTree. 
The former is to spliting the polynomial into two sub-polynomial and evaluate a polynomial at a given point.
The latter generates a Merkle tree root as a commitment.

In [ ]:
class PolynomialOperations:
    @staticmethod
    def split_polynomial(coeffs):
        g0 = coeffs[::2]  # Even coefficients
        g1 = coeffs[1::2]  # Odd coefficients
        return g0, g1

    @staticmethod
    def evaluate_polynomial(coeffs, x):
        return sum(c * x**i for i, c in enumerate(coeffs))

class MerkleTree:
    @staticmethod
    def commit(data):
        leaves = [sha256(bytes(str(d), 'utf-8')).digest() for d in data]
        while len(leaves) > 1:
            leaves = [sha256(leaves[i] + leaves[i + 1]).digest() for i in range(0, len(leaves), 2)]
        return leaves[0]

Now, we are prepared to define the protocol.

In [ ]:
 import numpy as np
from hashlib import sha256

class FRI:
    def __init__(self, domain, degree, randomness_source):
        self.domain = np.array(domain)
        self.degree = degree
        self.randomness_source = randomness_source
        self.commitments = []
        self.queries = []
        self.proofs = []

    def commit_phase(self, poly_coeffs):
        current_poly = poly_coeffs
        for _ in range(int(np.log2(self.degree))):
            # Split polynomial into g0 and g1
            g0, g1 = PolynomialOperations.split_polynomial(current_poly)

            # Verifier sends randomness
            v = self.randomness_source()

            # Compute next polynomial p_i
            current_poly = g0 + v * g1

            # Commit to p_i
            commitment = MerkleTree.commit(current_poly)
            self.commitments.append(commitment)

        # Final degree-0 polynomial
        self.proofs.append(current_poly[0])

    def query_phase(self, verifier_randomness):
        for r in verifier_randomness:
            # Query p0(r) and p0(-r)
            p0_r = PolynomialOperations.evaluate_polynomial(self.commitments[0], r)
            p0_minus_r = PolynomialOperations.evaluate_polynomial(self.commitments[0], -r)

            # Compute p1(r^2)
            r_squared = r**2
            g0_r2 = (p0_r + p0_minus_r) / 2
            g1_r2 = (p0_r - p0_minus_r) / (2 * r)
            p1_r2 = g0_r2 + self.randomness_source() * g1_r2

            # Check consistency
            self.queries.append((r, p0_r, p0_minus_r, p1_r2))

    def verify(self):
        for r, p0_r, p0_minus_r, p1_r2 in self.queries:
            # Compute g0(r^2) and g1(r^2) from p0
            g0_r2 = (p0_r + p0_minus_r) / 2
            g1_r2 = (p0_r - p0_minus_r) / (2 * r)

            # Compute expected p1(r^2)
            v = self.randomness_source()
            expected_p1_r2 = g0_r2 + v * g1_r2

            # Check consistency
            if not np.isclose(p1_r2, expected_p1_r2):
                return False
        return True

Below, we take an example function $f = 1+2x +3x^2+4x^3$ to show how FRI works.

In [ ]:
# Example usage
def random_element():
    return np.random.rand()

domain = [1, -1, 2, -2, 4, -4, 8, -8]
degree = 4
poly_coeffs = [1, 2, 3, 4]
fri = FRI(domain, degree, random_element)

# Commit phase
fri.commit_phase(poly_coeffs)

# Query phase
verifier_randomness = [1, 2]
fri.query_phase(verifier_randomness)

# Verify
assert fri.verify(), "Verification failed!"